In [1]:
import os
import re
import json
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt

import tensorflow as tf
print(tf.__version__)
import tensorflow_io as tfio
print(tfio.__version__)

from tensorflow.keras import layers
from tensorflow.keras import models

import wandb
from wandb.keras import WandbMetricsLogger

2023-03-09 21:14:46.154928: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 21:14:47.420268: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-03-09 21:14:47.420476: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or di

2.11.0
0.31.0


In [2]:
data_path = "../data/tfrecords"


def natural_keys(text):
    ""
    def atoi(text):
        return int(text) if text.isdigit() else text
    
    return [atoi(c) for c in re.split(r'(\d+)', text)]

tfrecords = sorted(glob(f"{data_path}/*.tfrec"), key=natural_keys)

In [3]:
train_tfrecords, valid_tfrecords = tfrecords[:19], tfrecords[19:]
print(len(train_tfrecords)+len(valid_tfrecords))

24


In [4]:
def parse_sequence(serialized_sequence):
    return tf.io.parse_tensor(
        serialized_sequence,
        out_type=tf.float32,
    )


def parse_tfrecord_fn(example):
    feature_description = {
        "n_frames": tf.io.FixedLenFeature([], tf.float32),
        "frames": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.int64),
    }
    
    return tf.io.parse_single_example(example, feature_description)


NUM_FRAMES = 16

def true_fn(frames, n_frames):
    num_left_frames = NUM_FRAMES - n_frames
    left_frames = tf.zeros(shape=(num_left_frames, 543, 3))
    frames = tf.concat([frames, left_frames], 0)

    return frames


def false_fn(frames):
    frames = tf.slice(
        frames,
        begin=[0,0,0],
        size=[NUM_FRAMES, 543, 3]
    )
    
    return frames


@tf.function
def preprocess_frames(frames, n_frames):
    """This is where different preprocessing logics will be experimented."""
    # nan to num
    frames = tf.where(tf.math.is_nan(frames), 0.0, frames)
    
    # sample frames
    frames = tf.cond(
        tf.less(n_frames, NUM_FRAMES),
        true_fn = lambda: true_fn(frames, n_frames),
        false_fn = lambda: false_fn(frames),
    )
    
    return frames


def parse_data(example):
    # Parse Frames
    n_frames = example["n_frames"]
    frames = tf.reshape(parse_sequence(example["frames"]), shape=(n_frames, 543, 3))
    frames = preprocess_frames(frames, n_frames)
    
    # Parse Labels
    label = tf.one_hot(example["label"], depth=250)

    return frames, label

In [5]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.data.TFRecordDataset(train_tfrecords)
valid_ds = tf.data.TFRecordDataset(valid_tfrecords)

train_ds = (
    train_ds
    .map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
    .shuffle(1024)
    .map(parse_data, num_parallel_calls=AUTOTUNE)
    .batch(128)
    .prefetch(AUTOTUNE)
)

valid_ds = (
    valid_ds
    .map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
    .map(parse_data, num_parallel_calls=AUTOTUNE)
    .batch(128)
    .prefetch(AUTOTUNE)
)

2023-03-09 21:14:56.422156: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-09 21:14:56.616676: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-09 21:14:56.617133: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-09 21:14:56.622000: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [14]:
tf.keras.backend.clear_session()

# def get_model():
#     inputs = layers.Input((32, 543, 3))
#     x = layers.ConvLSTM1D(16, 2)(inputs)
#     x = layers.GlobalAveragePooling1D()(x)
#     x = layers.Dropout(0.5)(x)
#     x = layers.Dense(250, activation="softmax")(x)
    
#     return models.Model(inputs, x)


def conv1d_lstm_block(inputs, filters):
    vector = tf.keras.layers.ConvLSTM1D(filters=32, kernel_size=8)(inputs)
    for f in filters:
        vector = tf.keras.layers.Conv1D(filters=f, kernel_size=8)(vector)
        vector = tf.keras.layers.MaxPooling1D()(vector)
    vector = tf.keras.layers.Dropout(0.3)(vector)
    return vector

def get_model():
    inputs = tf.keras.Input((NUM_FRAMES, 543, 3), dtype=tf.float32)

    # Features
    face_inputs = inputs[:, :, 0:468, :]
    left_hand_inputs = inputs[:, :, 468:489, :]
    pose_inputs = inputs[:, :, 489:522, :]
    right_hand_inputs = inputs[:, :,522:,:]

    face_vector = conv1d_lstm_block(face_inputs, [32, 64])
    left_hand_vector = conv1d_lstm_block(left_hand_inputs, [64])
    right_hand_vector = conv1d_lstm_block(right_hand_inputs, [64])
    pose_vector = conv1d_lstm_block(pose_inputs, [64])
    
    vector = tf.keras.layers.Concatenate(axis=1)([face_vector, left_hand_vector, right_hand_vector, pose_vector])
    vector = tf.keras.layers.Flatten()(vector)
    output = tf.keras.layers.Dense(250, activation="softmax")(vector)
    model = tf.keras.Model(inputs=inputs, outputs=output)
    
    return model


model = get_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 16, 543, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 16, 468, 3)  0           ['input_1[0][0]']                
 ingOpLambda)                                                                                     
                                                                                                  
 conv_lstm1d (ConvLSTM1D)       (None, 461, 32)      35968       ['tf.__operators__.getitem[0][0]'
                                                                 ]                            

In [15]:
model.compile(
    "adam",
    "binary_crossentropy",
    metrics=["acc"]
)

In [16]:
model.fit(
    train_ds,
    epochs=10,
    validation_data=valid_ds
)

Epoch 1/10
585/585 [==============================] - 211s 337ms/step - loss: 0.0301 - acc: 0.0059 - val_loss: 0.0250 - val_acc: 0.0248
Epoch 2/10
585/585 [==============================] - 183s 313ms/step - loss: 0.0235 - acc: 0.0449 - val_loss: 0.0216 - val_acc: 0.0952
Epoch 3/10
585/585 [==============================] - 182s 311ms/step - loss: 0.0209 - acc: 0.1206 - val_loss: 0.0194 - val_acc: 0.1843
Epoch 4/10
585/585 [==============================] - 183s 312ms/step - loss: 0.0190 - acc: 0.1978 - val_loss: 0.0181 - val_acc: 0.2439
Epoch 5/10
585/585 [==============================] - 182s 310ms/step - loss: 0.0178 - acc: 0.2544 - val_loss: 0.0171 - val_acc: 0.2846
Epoch 6/10
585/585 [==============================] - 181s 310ms/step - loss: 0.0170 - acc: 0.2915 - val_loss: 0.0175 - val_acc: 0.2739
Epoch 7/10
585/585 [==============================] - 181s 310ms/step - loss: 0.0164 - acc: 0.3195 - val_loss: 0.0173 - val_acc: 0.2855
Epoch 8/10
585/585 [============================

In [17]:
model.evaluate(valid_ds)

154/154 [==============================] - 10s 66ms/step - loss: 0.0159 - acc: 0.3494


[0.015903348103165627, 0.3493902385234833]

In [18]:
model.save("models/baseline")

INFO:tensorflow:Assets written to: baseline/assets


INFO:tensorflow:Assets written to: baseline/assets


## Inference

In [7]:
ROWS_PER_FRAME = 543  # number of landmarks per frame
SAMPLE_FILE = "train_landmark_files/2044/635217.parquet"

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

frames = load_relevant_data_subset(SAMPLE_FILE)

In [8]:
frames.shape

(7, 543, 3)

In [29]:
class DataPreprocessing(tf.Module):
    def __init__(self, num_frames=NUM_FRAMES, name=None):
        super().__init__(name=name)
        self.num_frames = num_frames
        
    def true_fn(self, frames, n_frames):
        num_left_frames = self.num_frames - n_frames
        left_frames = tf.zeros(shape=(num_left_frames, 543, 3))
        frames = tf.concat([frames, left_frames], 0)

        return frames

    def false_fn(self, frames):
        frames = tf.slice(
            frames,
            begin=[0,0,0],
            size=[self.num_frames, 543, 3]
        )

        return frames
    
    def shape_list(self, tensor):
        """
        Deal with dynamic shape in tensorflow cleanly.
        Args:
            tensor (`tf.Tensor` or `np.ndarray`): The tensor we want the shape of.
        Returns:
            `List[int]`: The shape of the tensor as a list.
        """
        if isinstance(tensor, np.ndarray):
            return list(tensor.shape)

        dynamic = tf.shape(tensor)

        if tensor.shape == tf.TensorShape(None):
            return dynamic

        static = tensor.shape.as_list()

        return [dynamic[i] if s is None else s for i, s in enumerate(static)]

    def __call__(self, frames):
        n_frames, _, _ = self.shape_list(frames)
        
        # nan to num
        frames = tf.where(tf.math.is_nan(frames), 0.0, frames)

        # sample frames
        frames = tf.cond(
            tf.less(n_frames, NUM_FRAMES),
            true_fn = lambda: true_fn(frames, n_frames),
            false_fn = lambda: false_fn(frames),
        )

        return tf.expand_dims(frames, axis=0)

In [30]:
model = tf.keras.models.load_model("models/baseline")
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 16, 543, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 16, 468, 3)  0           ['input_1[0][0]']                
 ingOpLambda)                                                                                     
                                                                                                  
 conv_lstm1d (ConvLSTM1D)       (None, 461, 32)      35968       ['tf.__operators__.getitem[0][0]'
                                                                 ]                            

In [35]:
class TFLiteModel(tf.keras.Model):
    """
    TensorFlow Lite model that takes input tensors and applies:
        – a preprocessing model
        – the ASL model 
    """

    def __init__(self, asl_model):
        """
        Initializes the TFLiteModel with the specified feature generation model and main model.
        """
        super(TFLiteModel, self).__init__()

        # Load the feature generation and main models
        self.prep_inputs = DataPreprocessing()
        self.model = model
    
    @tf.function(input_signature=[tf.TensorSpec(shape=[None, 543, 3], dtype=tf.float32, name='inputs')])
    def call(self, inputs):
        """
        Applies the feature generation model and main model to the input tensors.

        Args:
            inputs: Input tensor with shape [batch_size, 543, 3].

        Returns:
            A dictionary with a single key 'outputs' and corresponding output tensor.
        """
        x = self.prep_inputs(tf.cast(inputs, dtype=tf.float32))
        outputs = self.model(x)[0, :]

        # Return a dictionary with the output tensor
        return {'outputs': outputs}

tflite_keras_model = TFLiteModel(model)
demo_output = tflite_keras_model(load_relevant_data_subset(SAMPLE_FILE))["outputs"]
np.argmax(demo_output.numpy(), axis=-1)

30

In [38]:
keras_model_converter = tf.lite.TFLiteConverter.from_keras_model(tflite_keras_model)
tflite_model = keras_model_converter.convert()
with open('models/model.tflite', 'wb') as f:
    f.write(tflite_model)
!zip submission.zip models/model.tflite

import tflite_runtime.interpreter as tflite

interpreter = tflite.Interpreter("models/model.tflite")
found_signatures = list(interpreter.get_signature_list().keys())
prediction_fn = interpreter.get_signature_runner("serving_default")

output = prediction_fn(inputs=load_relevant_data_subset(SAMPLE_FILE))
sign = np.argmax(output["outputs"])

print(sign)

INFO:tensorflow:Assets written to: /tmp/tmp0khadsct/assets


INFO:tensorflow:Assets written to: /tmp/tmp0khadsct/assets
2023-03-09 22:16:07.988475: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-03-09 22:16:07.988535: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-03-09 22:16:07.989575: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp0khadsct
2023-03-09 22:16:08.035732: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-03-09 22:16:08.035782: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp0khadsct
2023-03-09 22:16:08.186827: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-03-09 22:16:08.218611: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-03-09 22:16:08.457056: I tensorflow/cc/saved_model/loader.cc:213] Running initializatio

  adding: models/model.tflite (deflated 9%)
30
